In [15]:
import os
import h5py

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

import torch
import torch.nn as nn
import torchvision
from torch.utils import data
import heidel_utils
from importlib import reload # reload 
reload(heidel_utils)


<module 'heidel_utils' from '/Users/svenkerstjens/msc-thesis/MSc-thesis/heidel_utils.py'>

In [6]:
nb_inputs  = 700
nb_hidden  = 200
nb_outputs = 20

time_step = 1e-3
nb_steps = 100
max_time = 1.4

batch_size = 256
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'device: {device}')
else:
    device = torch.device("cpu")
    print(f'device: {device}')

device: cpu


In [16]:
# Here we load the Dataset
cache_dir = os.path.expanduser("~/data")
cache_subdir = "hdspikes"
heidel_utils.get_shd_dataset(cache_dir, cache_subdir)

train_file = h5py.File(os.path.join(cache_dir, cache_subdir, 'shd_train.h5'), 'r')
test_file = h5py.File(os.path.join(cache_dir, cache_subdir, 'shd_test.h5'), 'r')

x_train = train_file['spikes']
y_train = train_file['labels']
x_test = test_file['spikes']
y_test = test_file['labels']

Available at: /Users/svenkerstjens/data/hdspikes/shd_train.h5
Available at: /Users/svenkerstjens/data/hdspikes/shd_test.h5


In [18]:
def sparse_data_generator_from_hdf5_spikes(X, y, batch_size, nb_steps, nb_units, max_time, shuffle=True):
    """ This generator takes a spike dataset and generates spiking network input as sparse tensors.

    Args:
        X: The data ( sample x event x 2 ) the last dim holds (time,neuron) tuples
        y: The labels
    """

    labels_ = np.array(y,dtype=np.int)
    number_of_batches = len(labels_)//batch_size
    sample_index = np.arange(len(labels_))

    # compute discrete firing times
    firing_times = X['times']
    units_fired = X['units']

    time_bins = np.linspace(0, max_time, num=nb_steps)

    if shuffle:
        np.random.shuffle(sample_index)

    total_batch_count = 0
    counter = 0
    while counter<number_of_batches:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]

        coo = [ [] for i in range(3) ]
        for bc,idx in enumerate(batch_index):
            times = np.digitize(firing_times[idx], time_bins)
            units = units_fired[idx]
            batch = [bc for _ in range(len(times))]

            coo[0].extend(batch)
            coo[1].extend(times)
            coo[2].extend(units)

        i = torch.LongTensor(coo).to(device)
        v = torch.FloatTensor(np.ones(len(coo[0]))).to(device)

        X_batch = torch.sparse.FloatTensor(i, v, torch.Size([batch_size,nb_steps,nb_units])).to(device)
        y_batch = torch.tensor(labels_[batch_index],device=device)

        yield X_batch.to(device=device), y_batch.to(device=device)

        counter += 1

In [25]:
import time
from LIFlayer import OneHiddenModel
model = OneHiddenModel(in_channels=nb_inputs,hidden_channels=nb_hidden,out_channels=nb_outputs,batch_size=batch_size,W=None)

ce_loss = torch.nn.CrossEntropyLoss()

params = model.parameters()
opt = torch.optim.Adam(params, lr=1e-4, betas=[0., .95]) #lr is the learning rate
decay = .9

epochs = 10

for epoch in range(epochs):
    tic = time.time()
    avg_loss = 0
    model.train()
    sum_acc=0
    #batches
    for x,y in sparse_data_generator_from_hdf5_spikes(x_train, y_train, batch_size, nb_steps, nb_units=nb_inputs, max_time=max_time, shuffle=True):
        tik = time.time()
        model.init_states()

        Sprobe = torch.zeros((batch_size,model.out_channels))

        #timesteps
        for n in range(nb_steps):
            out_state = model(x.to_dense()[:,n])

            #add decay for leakiness
            #collect spikes over time
            Sprobe = decay * Sprobe + out_state.S


        prediction = Sprobe
        #if i==0:
        #    print(prediction)

        #accuracy = val_accuracy(prediction,y_train[train_batch_ids[i]])

        #class labels only
        loss = ce_loss(prediction,y)

        print(f'batchloss:  {loss}')

        #tonic & torch neuromorphic

        loss.backward()
        opt.step()
        opt.zero_grad()
        #model.init_mod_weights(W2)
        #sum_acc = sum_acc + accuracy
        #sum_loss = sum_loss + loss
        count = 0
        avg_loss = ((avg_loss * count) + loss)/(count+1)
        count += 1
    #avg_train_acc = sum_acc/(len(train_batch_ids))
    #avg_loss = sum_loss/(len(train_batch_ids))
    #if i == 3:/'
    #loss_hist = loss_hist + [float(avg_loss)]
    #acc_hist = acc_hist + [float(avg_train_acc)]
        tok = time.time()
        print(f'time for batch: {tok-tik}')
    toc = time.time()
    print(f'time for epoch {epoch}: {toc-tic}')
    if epoch%1==0:
        print(f'epoch {epoch}: \n loss: {avg_loss}')
        #print(f'train_acc: {avg_train_acc}')
    #val_acc,_,_ = validation_acc(X_test,y_test,model,test_batch_ids)
    #val_acc_hist = val_acc_hist + [val_acc]
    #print(f'val_acc: {val_acc}')



ImportError: cannot import name 'OneHiddenModel' from 'LIFlayer' (/Users/svenkerstjens/msc-thesis/MSc-thesis/LIFlayer.py)